This notebook uses corrected $R_{eff}$ estimation data from Tamas Ferenci, and combines it with latest Hungarian data
about number of confirmed cases and test positivity downloaded directly from the webpage of Tamas Ferenci
(which is downloaded from the Johns Hopkins dashboard).

In [ ]:
from datetime import datetime, timedelta
import os
import sys

import pandas as pd
import matplotlib.pyplot as plt

sys.path.append("../")
from src.dataloader import PROJECT_PATH

In [ ]:
plt.rcParams["font.size"] = 18

In [ ]:
# reading r_eff estimation data from Tamas Ferenci
# institutional and imported cases are excluded in Spring and Summer 2020
# source: e-mailed by the author

ferenci_corrected = pd.read_csv(
    os.path.join(PROJECT_PATH, 'data/r_eff_estimations.csv'),
    sep=";",
    header=None,
    index_col=0
)

ferenci_corrected["date"] = ferenci_corrected[4]\
    .map(lambda i: (datetime(2020,3,3)+timedelta(days=i)).strftime('%Y-%m-%d'))
ferenci_corrected["r_eff"] = ferenci_corrected[1].map(lambda s: float(s.replace(',','.')))
ferenci_corrected["ci_lower"] = ferenci_corrected[2].map(lambda s: float(s.replace(',','.')))
ferenci_corrected["ci_upper"] = ferenci_corrected[3].map(lambda s: float(s.replace(',','.')))
ferenci_corrected["method"] = ferenci_corrected[5]
ferenci_corrected.drop([1,2,3,4,5],axis=1,inplace=True)

ferenci_corrected.head()

In [ ]:
# latest estimations downloaded directly from the webpage of Tamas Ferenci
# has to be downloaded by hand...
# https://research.physcon.uni-obuda.hu/COVID19MagyarEpi/
# Ctrl+C / Ctrl + V whole table from the "Reprodukcios szam becslese" ful > data/ferenci_r_eff.csv

In [ ]:
ferenci = pd.read_csv(os.path.join(PROJECT_PATH, 'data/ferenci_r_eff.csv'),
                      sep="\t",header=None)
ferenci.columns = ["method","date","junk"]
ferenci["junk"] = ferenci["junk"].map(lambda s: s.replace("("," ").replace(")"," ").replace("-"," "))
ferenci["r_eff"] = ferenci["junk"].map(lambda s: float(s.split(" ")[0]))
ferenci["ci_lower"] = ferenci["junk"].map(lambda s: float(s.split(" ")[2]))
ferenci["ci_upper"] = ferenci["junk"].map(lambda s: float(s.split(" ")[3]))
ferenci["date"] = ferenci["date"].map(lambda d: datetime.strptime(d,"%m/%d/%Y").strftime("%Y-%m-%d"))
ferenci.drop(["junk"],axis=1,inplace=True)

In [ ]:
temp = []
for m in ferenci_corrected["method"].unique():
    df1 = ferenci_corrected[ferenci_corrected["method"]==m]
    df2 = ferenci[ferenci["method"]==m]
    temp.extend([df1,df2[df2["date"]>df1["date"].max()]])

In [ ]:
data = pd.concat(temp)

In [ ]:
# number of cases and test positivity downloaded directly from the webpage of Tamas Ferenci
file_path = 'https://research.physcon.uni-obuda.hu/COVID19MagyarEpi/session/' + \
            'fe81b58c8fa141a13cc35b7c868a15c0/download/testpositivityTabDlCSV?w='
casenum = pd.read_csv(file_path, sep=";")
casenum["pos"] = casenum["Tesztpozitivitás [%]"].map(lambda s: float(str(s).replace(",", ".")))
casenum["date"] = casenum["Dátum"]

# joining case number and test positivity to the previous results
data["pos"] = data["date"].map(dict(zip(casenum["date"].map(str), casenum["pos"])))
data["casenum"] = data["date"].map(dict(zip(casenum["date"].map(str), casenum["Napi esetszám [fő/nap]"])))

In [ ]:
# test view
data

In [ ]:
# exporting to csv
data.to_csv(
    os.path.join(PROJECT_PATH, 'data/reference_r_eff.csv'))

In [ ]:
# test figure
plt.figure(figsize=(20,10))
plt.plot(data[data["method"]=="Cori"]["date"].map(
    lambda d: datetime.strptime(d,"%Y-%m-%d")),data[data["method"]=="Cori"]["r_eff"])
plt.grid()